In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Project-X-Lip-Reading

/content/drive/MyDrive/Project-X-Lip-Reading


In [3]:
import pickle
import time
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, LSTM, Dense, Dropout, Bidirectional, MaxPool3D, MaxPooling3D
from tensorflow.keras.layers import Activation, Reshape, SpatialDropout3D, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.optimizers import Adam

In [4]:
with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/train/X_train.pkl', 'rb') as f:
   X_train = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/train/y_train.pkl', 'rb') as f:
   y_train = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/test/X_test.pkl', 'rb') as f:
   X_test = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/test/y_test.pkl', 'rb') as f:
   y_test = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/val/X_val.pkl', 'rb') as f:
   X_val = pickle.load(f)

with open('/content/drive/MyDrive/Project-X-Lip-Reading/Dataset/val/y_val.pkl', 'rb') as f:
   y_val = pickle.load(f)

In [5]:
def build_3d_cnn_model(input, num_classes, learning_rate = 0.01, dropout_rate = 0.5):

    model = Sequential()

    # 1st Layer group
    model.add(Conv3D(128, 3, input_shape=input, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((2,2,2)))

    # 2nd Layer group
    model.add(Conv3D(256, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((2,2,2)))

    # 3rd Layer group
    model.add(Conv3D(75, 3, padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPool3D((2,2,2)))

    # Reshape
    model.add(Reshape((75, 2*12*12)))

    # Bi-LSTM Layers
    model.add(Bidirectional(LSTM(32, kernel_initializer='Orthogonal', return_sequences=True)))
    model.add(Dropout(dropout_rate))

    model.add(Bidirectional(LSTM(32, kernel_initializer='Orthogonal', return_sequences=True)))
    model.add(Dropout(dropout_rate))

    # Flatten Layer
    model.add(Flatten())

    # Dense Layer
    model.add(Dense(num_classes, activation='softmax'))

    # Model Compilation
    model.compile(optimizer=Adam(learning_rate = learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [6]:
# Tuning Hyperparameters
input = (22, 100, 100, 1)
num_classes = 10
learning_rate = 0.001
dropout_rate = 0.5

In [7]:
# Build Model
model = build_3d_cnn_model(input, num_classes, learning_rate, dropout_rate)

In [8]:
# Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 22, 100, 100, 12   3584      
                             8)                                  
                                                                 
 activation (Activation)     (None, 22, 100, 100, 12   0         
                             8)                                  
                                                                 
 max_pooling3d (MaxPooling3  (None, 11, 50, 50, 128)   0         
 D)                                                              
                                                                 
 conv3d_1 (Conv3D)           (None, 11, 50, 50, 256)   884992    
                                                                 
 activation_1 (Activation)   (None, 11, 50, 50, 256)   0         
                                                        

In [ ]:
# Early Stopping
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor="val_loss", patience=4)

# Model Training
import time
t1 = time.time()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size = 32, callbacks = [early_stop])
t2 = time.time()
print(f"Training time : {t2 - t1} sec")

Epoch 1/100
4/4 [==============================] - 763s 181s/step - loss: 1.9170 - accuracy: 0.2150 - val_loss: 1.1440 - val_accuracy: 0.3571
Epoch 2/100
1/4 [======>.......................] - ETA: 10:31 - loss: 1.1188 - accuracy: 0.4375

In [ ]:
# Visualizing the training and validation loss
plt.plot(history.history['loss'], 'blue', label = 'loss')
plt.plot(history.history['val_loss'], 'red', label = 'validation_loss')
plt.title("Total Loss vs Total Validation Loss")
plt.legend()
plt.show()

In [ ]:
# Visualizing the training and validation accuracy
plt.plot(history.history['accuracy'], 'blue', label = 'accuracy')
plt.plot(history.history['val_accuracy'], 'red', label = 'validation_accuracy')
plt.title("Total Accuracy vs Total Validation Accuracy")
plt.legend()
plt.show()

In [ ]:
# Model Evaluation
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# Saving Model
model.save('/content/drive/MyDrive/Project-X-Lip-Reading/Model Architecture/Saved Model/3D_CNN_Bi-LSTM.keras')